In [1]:
%dependency /add com.microsoft.semantic-kernel:semantickernel-api:0.2.13-alpha
%dependency /add com.microsoft.semantic-kernel:semantickernel-bom:0.2.13-alpha
%dependency /add com.microsoft.semantic-kernel:semantickernel-parent:0.2.13-alpha
%dependency /add com.microsoft.semantic-kernel:semantickernel-connectors-ai-openai:0.2.13-alpha
%dependency /add com.microsoft.semantic-kernel:semantickernel-planners:0.2.13-alpha
%dependency /add com.microsoft.semantic-kernel:semantickernel-core:0.2.13-alpha

%dependency /override com.azure:azure-core:1.43.0
%dependency /override org.slf4j:slf4j-api:2.0.7

%dependency /override com.google.code.gson:gson:2.8.9
%dependency /override com.google.protobuf:protobuf-java:3.23.2
%dependency /override com.google.errorprone:error_prone_annotations:2.18.0
%dependency /override com.google.guava:guava:32.0.1-android

// %dependency /add io.qdrant:client:1.8.0


%dependency /add org.slf4j:slf4j-jdk14:2.0.12

%dependency /resolve

Adding dependency com.microsoft.semantic-kernel:semantickernel-api:0.2.13-alpha
Adding dependency com.microsoft.semantic-kernel:semantickernel-bom:0.2.13-alpha
Adding dependency com.microsoft.semantic-kernel:semantickernel-parent:0.2.13-alpha
Adding dependency com.microsoft.semantic-kernel:semantickernel-connectors-ai-openai:0.2.13-alpha
Adding dependency com.microsoft.semantic-kernel:semantickernel-planners:0.2.13-alpha
Adding dependency com.microsoft.semantic-kernel:semantickernel-core:0.2.13-alpha
Adding dependency override com.azure:azure-core:1.43.0
Adding dependency override org.slf4j:slf4j-api:2.0.7
Adding dependency override com.google.code.gson:gson:2.8.9
Adding dependency override com.google.protobuf:protobuf-java:3.23.2
Adding dependency override com.google.errorprone:error_prone_annotations:2.18.0
Adding dependency override com.google.guava:guava:32.0.1-android
Adding dependency org.slf4j:slf4j-jdk14:2.0.12
Solving dependencies
Found dependencies count: 41
Add to classpath:

In [2]:
import com.azure.ai.openai.OpenAIAsyncClient;
import com.microsoft.semantickernel.Kernel;
import com.microsoft.semantickernel.SKBuilders;
import com.microsoft.semantickernel.exceptions.ConfigurationException;
import com.microsoft.semantickernel.orchestration.SKContext;
import com.microsoft.semantickernel.skilldefinition.annotations.DefineSKFunction;
import com.microsoft.semantickernel.skilldefinition.annotations.SKFunctionInputAttribute;
import com.microsoft.semantickernel.textcompletion.TextCompletion;

In [3]:
import com.microsoft.semantickernel.chatcompletion.ChatCompletion;
import com.microsoft.semantickernel.chatcompletion.ChatHistory;

In [4]:
import com.azure.ai.openai.OpenAIAsyncClient;
import com.azure.ai.openai.OpenAIClientBuilder;
import com.azure.core.credential.AzureKeyCredential;
import com.azure.core.credential.KeyCredential;

In [5]:
final OpenAIAsyncClient client = new OpenAIClientBuilder()
        .credential(new AzureKeyCredential("Your Azure OpenAI Service Key"))
        .endpoint("Your Azure OpenAI Service EndPoint")
        .buildAsyncClient();

In [6]:
public static class WeatherSkill {

    @DefineSKFunction(description = "get current Weather from city")
    public String getCurrentWeatherFromCity(
        @SKFunctionInputAttribute(description = "input city to get current weather") String city) {
        return city + "’s weather is 30 celsius degree ";
    }
}

In [7]:
public static class WriteSkill {

    @DefineSKFunction(description = "write dressing tips for weather")
    public String writeDressingTips(
        @SKFunctionInputAttribute(description = "input city weather") String weather) {

        OpenAIAsyncClient aoaiclient = new OpenAIClientBuilder()
                .credential(new AzureKeyCredential("Your Azure OpenAI Service API Key"))
                .endpoint("Your Azure OpenAI Service Endpoint")
                .buildAsyncClient();

        

        TextCompletion textCompletion = SKBuilders.textCompletion()
                                                            .withModelId("Your Azure OpenAI Service gpt-3.5-instruct deployment model")
                                                            .withOpenAIClient(aoaiclient)
                                                            .build();

        Kernel kernel = SKBuilders.kernel().withDefaultAIService(textCompletion).build();
        
        String functionDefinition = """
            {{$input}}
            Write dressing tips in Chinese about the content above
        """.stripIndent();


        var weatherFunction = SKBuilders
            .completionFunctions()
            .withKernel(kernel)
            .withPromptTemplate(functionDefinition)
            .withRequestSettings(
                SKBuilders.completionRequestSettings()
                    .temperature(0.7)
                    .topP(1)
                    .maxTokens(1024)
                    .build())
            .build();
        
        
        var result = weatherFunction.invokeAsync(weather).block();
        return result.getResult();
    }
}

In [8]:


var kernel = SKBuilders.kernel()
                .withDefaultAIService(SKBuilders.chatCompletion()
                        .withOpenAIClient(client)
                        .withModelId("Your Azure OpenAI Service gpt-3.5-turbo-16k deployment model")
                        .build())
                .build();



In [9]:
import java.nio.file.Path;

In [10]:
kernel.importSkill(new WeatherSkill(), "weather");
kernel.importSkill(new WriteSkill(), "writer");



com.microsoft.semantickernel.skilldefinition.FunctionCollection@36caa23b

In [11]:

import com.microsoft.semantickernel.planner.sequentialplanner.SequentialPlanner;
import com.microsoft.semantickernel.planner.sequentialplanner.SequentialPlannerRequestSettings;


In [12]:

private static SequentialPlanner initPlanner(Kernel kernel, int maxTokens) {
    return new SequentialPlanner(kernel, new SequentialPlannerRequestSettings(
            null,
            100,
            Set.of(),
            Set.of(),
            Set.of(),
            maxTokens,
            false
    ), null);
}

In [13]:
SequentialPlanner planner = initPlanner(kernel, 1024);


In [14]:
var goal = "get Guangzhou current weather ,then write dressing tips about it";

In [15]:

var plan = planner.createPlanAsync(goal).block();


In [16]:

plan.invokeAsync().block().getResult();

广州的天气是30摄氏度
以下是穿衣建议：

1. 室外温度高，建议选择轻薄、透气的衣服，如短袖衬衫、短裙等。

2. 带一把遮阳伞或帽子，防止阳光直射。

3. 选择浅色衣服，能够反射太阳光，减少热量吸收。

4. 带一件薄外套，防止进入冷气房时感觉过冷。

5. 注意防晒，涂抹防晒霜，防止晒伤皮肤。

6. 避免穿太紧身的衣服，选择宽松舒适的服装。

7. 多喝水，保持水分补充，防止中暑。

8. 出门前注意检查天气预报，避免遇到突然的降温或雨天。